In [1]:
import boto3
import pandas as pd
import awswrangler as wr

In [2]:
! aws sso login --profile Stellaralgo-DataScienceAdmin

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

RDMK-VBJM
Successully logged into Start URL: https://stellaralgo.awsapps.com/start


In [3]:
session = boto3.setup_default_session(profile_name='Stellaralgo-DataScienceAdmin')
client = boto3.client('redshift')

cluster = 'qa-app'
dbname = 'datascience'
schema = 'ds'
table = 'dummytable'
    
conn = wr.data_api.redshift.connect(
    cluster_id = cluster,
    database = dbname,
    db_user = "admin"
)

In [20]:
df_data_dictionary = pd.read_csv('./data_dictionary.csv')
df_data_dictionary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Model        84 non-null     object
 1   Feature Raw  84 non-null     object
 2   Feature      84 non-null     object
 3   Data Type    84 non-null     object
 4   Source       82 non-null     object
 5   Engineered   84 non-null     bool  
 6   Description  84 non-null     object
dtypes: bool(1), object(6)
memory usage: 4.1+ KB


In [21]:
fields = f"""
    INSERT INTO datascience.ds.datadictionary (
        feature,
        featureraw,
        datatype,
        source,
        engineered,
        description
    ) VALUES """

values_list = []
for i, row in df_data_dictionary.iterrows():
    
    values = f"""(
        '{row["Feature"]}',
        '{row["Feature Raw"]}',
        '{row["Data Type"]}',
        '{row["Source"]}',
        {row["Engineered"]},
        '{row["Description"]}'
    )"""
    
    values_list.append(values)
        
insert_statement = fields + ",".join(values_list)+";"
 
wr.data_api.redshift.read_sql_query(
    sql = insert_statement, 
    con = conn
)

""
